# Emissions RAG Orchestrator

This notebook implements an end-to-end pipeline that converts PDFs into structured text and extracts Scope 1, Scope 2, and Scope 3 emissions data.

The process has two stages:

**Stage 1 — PDF → Clean TXT (layout-aware extraction)**  
Each page is analysed, classified by layout, and routed to the correct extractor (reader, OCR, or vision).  
The result is one audited `.txt` per PDF where tables, scanned pages, and complex layouts are correctly reconstructed.

**Stage 2 — TXT → Chunks → Embeddings → Answers**  
The generated `.txt` files are chunked, embedded, and used to answer structured emissions questions through semantic retrieval.

This design ensures the AI works on clean, layout-preserved text and avoids sending simple pages to expensive models.



## Environment, Imports, Data Loading, and LLM Client Setup

In [2]:
# =========================
# Standard library
# =========================
import os
import sys
import csv
import json
import time
import re
import shutil
import logging
import traceback
import matplotlib.pyplot as plt
from uuid import uuid4
from datetime import datetime
from pathlib import Path
from pprint import pprint
from urllib.parse import urlparse, parse_qs
from concurrent.futures import ThreadPoolExecutor, as_completed
import importlib

# =========================
# Third-party libraries
# =========================
import fitz
import boto3
import requests as req
import pandas as pd
import psycopg2
import psycopg2.extras
import tiktoken
from bs4 import BeautifulSoup
from dotenv import load_dotenv
from flask import request, jsonify, Blueprint
from openai import OpenAI
from pypdf import PdfReader
from google import genai
from google.genai import types

# =========================
# Project modules
# =========================
import metrics3_utilities as m3
import kadoa_functions as kadoa
import esg_ai_pipeline_functions as esg
import pdf_processing_functions as transformer

# =========================
# Environment
# =========================
load_dotenv()

# Reload during notebook/dev work
importlib.reload(esg)
importlib.reload(kadoa)
importlib.reload(m3)
importlib.reload(transformer)

# =========================
# Paths & questions
# =========================
path = Path.cwd()

#questions_path = path / "questions-lists-bailey.xlsx"
csv_path = path / "questions.csv"

#new_questions = pd.read_excel(questions_path, sheet_name="Fernando")["Questions"]
questions = pd.read_csv(csv_path)

#questions["content"] = new_questions
questions = questions[0:3]

# =========================
# LLM Clients
# =========================
genai_client = genai.Client(
    api_key=os.getenv("GEMINI_API_KEY")
)

open_ai_client = OpenAI(
    organization=os.getenv("OPENAI_ORG_ID"),
    project=os.getenv("OPENAI_PROJECT_ID"),
    api_key=os.getenv("OPENAI_API_KEY")
)

# =========================
# Defining Folders
# =========================
pdf_folder = Path(os.getenv("FILINGS_PDF"))
txt_folder = Path(os.getenv("FILINGS_TXT"))

## PDF Processing and Output Generation
Runs the PDF pipeline over the input folder, classifies each page, applies the appropriate extraction method, and saves the consolidated text outputs to the results folder.

##### Building a list wih the stats of each company

In [57]:

# Build company_stats for ALL PDFs
company_stats = {}

for f in os.listdir(pdf_folder):
    if not f.lower().endswith(".pdf"):
        continue

    company = os.path.splitext(f)[0]
    pdf_path = pdf_folder / f

    size_bytes = pdf_path.stat().st_size

    with fitz.open(pdf_path) as doc:
        company_stats[company] = {
            "pages": doc.page_count,
            "file_size_bytes": size_bytes
        }

total_pages = sum(
    stats["pages"] for stats in company_stats.values()
)
total_pages

103748

##### Companies in Division 10 over 10M revenue

In [58]:
filtered_companies = (
    pd.read_excel(Path.cwd() / "div_comp_over_10M_rev.xlsx")["Companynumber"]
    .astype(str)
    .to_list()
)

# total pages using stats built from the folder
total_pages = sum(
    company_stats[c]["pages"]
    for c in filtered_companies
    if c in company_stats
)

print(f"number of companies: {len(filtered_companies)}")
print(f"total pages: {total_pages}")

number of companies: 1341
total pages: 35270


In [59]:
#selected_companies = filtered_companies[800:1341] 800 to 1100 already done
selected_companies = filtered_companies[0:800]
total_pages = sum(
    company_stats[c]["pages"]
    for c in selected_companies
    if c in company_stats
)
print(f"number of companies: {len(selected_companies)}")
print(f"total pages: {total_pages}")

number of companies: 800
total pages: 20340


In [62]:
transformer.process_pdfs(pdf_folder, txt_folder, companies=selected_companies)

Skipping 00047601 (already processed)
Skipping 00050639 (already processed)
Skipping 00052457 (already processed)
Skipping 00055762 (already processed)
Skipping 00079590 (already processed)
Skipping 00104785 (already processed)
Skipping 00111524 (already processed)
Skipping 00112075 (already processed)
Skipping 00124503 (already processed)
Skipping 00132872 (already processed)
Skipping 00144133 (already processed)
Skipping 07292136 (already processed)
Skipping 07592875 (already processed)
Skipping 07735102 (already processed)
Starting processing at 2026-02-17 12:19:29
Companies to process: 707
Processing: 00062904.pdf
Saved: 00062904.pdf
Progress: Companies 0.1% | Pages 2.0%
Processing: 00145472.pdf
Saved: 00145472.pdf
Progress: Companies 0.3% | Pages 2.1%
Processing: 00147624.pdf
Saved: 00147624.pdf
Progress: Companies 0.4% | Pages 2.3%
Processing: 00155256.pdf
Saved: 00155256.pdf
Progress: Companies 0.6% | Pages 2.6%
Processing: 00156737.pdf
Saved: 00156737.pdf
Progress: Companies 0.

## RAG Execution and Emissions Extraction

Iterates over each extracted TXT file and runs the RAG pipeline for that company to retrieve Scope 1, 2, and 3 emissions, and cleans the workspace after each run.

In [ ]:
companies = [
    p.stem.split("_")[0]
    for p in txt_folder.glob("*.txt")
]

txt_companies = list(companies)[0:1]

['00047601']

In [ ]:
instructions = esg.instructions

# --- read processed companies from Excel ---
excel_path = Path.cwd() / "Emissions Results (Market based).xlsx"

processed_companies = set(
    pd.read_excel(
        excel_path,
        dtype={"company_number": str}
    )["company_number"]
    .str.lstrip("'")
    .str.zfill(8)
    .unique()
)

for txt_file in txt_folder.glob("*.txt"):
    company_number = txt_file.stem.split("_")[0]

    # only companies you want to run
    if company_number not in txt_companies:
        continue

    # skip already processed
    if company_number in processed_companies:
        continue

    print(f"Running for company: {company_number}")

    esg.ai_run_and_save_scores_locally(
        open_ai_client,
        str(txt_file),
        company_number,
        instructions,
        questions,
        use_chunking=True
    )

FileNotFoundError: [Errno 2] No such file or directory: 'c:\\Users\\FernandoMaldondoTheD\\FernandoGit\\rag-fernando-testing\\Emissions Results (Market based).xlsx'